# PINN Solver for Hernquist profile

## 1. Introduction
We start from the Hernquist density profile $$\rho(r) = \dfrac{M}{2\pi}\dfrac{a}{r}\dfrac{1}{(r+a)} $$

For such a potential, the Poisson equation reads:
$$\nabla ^2 \phi = 4\pi G \rho$$

$$ \Leftrightarrow \dfrac{1}{r^2} \dfrac{\partial}{\partial r}\left(r^2 \dfrac{\partial \phi}{\partial r}\right) = 4\pi G \left[\dfrac{M}{2\pi}\dfrac{a}{r}\dfrac{1}{(r+a)}\right]$$

Given that the density-potential pair depends solely on the radial coordinate $r$, the equation reduces to
$$  \dfrac{1}{r} \dfrac{d}{d r}\left(r^2 \dfrac{d \phi}{d r}\right) = \dfrac{2GMa}{a^3(\frac{r}{a}+1)^3}$$


Setting $r \rightarrow \frac{r}{a}$, we get
$$  \dfrac{d}{d r}\left(r^2 \dfrac{d \phi}{d r}\right) = \dfrac{2GM}{a}\dfrac{r}{(r+1)^3}$$


Finally, in the case of the Hernquist profile, the Poisson equation can be written such as:

$$ \boxed{\dfrac{d}{d r}\left(r^2 \dfrac{d \phi}{d r}\right) = \dfrac{2r}{(r+1)^3}}$$

where we have set $\frac{GM}{a}$ to unity.

By [integrating Poisson's equation](/analytical_potential_hernquist.ipynb), we find for this non dimensional equation :

$$ \Phi(s) = -\dfrac{1}{s + 1} $$ where $s=\dfrac{r}{a}$.

## 2. Implementation of the Physics-Informed Neural Network

We implement a simple neural network with one hidden layer (to start with), which outputs a single value. The latter value corresponds to the amplitude of the field at the point of interest. 

### 2.1 The Neural Network

We use here a Feedforward Neural Network (FFNN). The input layer takes one co-location point as an input. It has one hidden layer composed of 100 neurons, and outputs the amplitude of the field at this point. The neural network is here used as a universel function approximator. We use for the activation function the hyperbolic tangent. 

Several architectures will be tried out for the FFNN: 

|Number of layers| Number of Neurons (per layer) | Activation function  | Article  | 
|----------------|-------------------------------|----------------------|----------|
|   2-4 layers   |   32 and 50 neurons           |                      |          |
|   5-8 layers   |   250 neurons                 |                      |          |
|   9+  layers   |   20 neurons                  |                      |          |

### 2.2 The Loss Function

As stated, the model is trained by minimizing the loss function. In the case of Physics-Informed Neural Networks (PINNs), the loss function can be written in a general way as :

$$ \mathcal{L}(\theta) = \omega_{\mathcal{F}}\mathcal{L}_{\mathcal{F}}(\theta) + \omega_{\mathcal{B}}\mathcal{L}_{\mathcal{B}}(\theta) + \omega_{\mathcal{d}}\mathcal{L}_{data}(\theta)$$ 
where $\theta$ are the parameters of the neural network, and 
$$\mathcal{L}_{\mathcal{F}}(\theta) = MSE_{\mathcal{F}} = \dfrac{1}{N_c}\sum^{N_c}_{i=1} \left\|\mathcal{F}(\hat{u}_{\theta}(z_i)) - f(z_i)\right\|^2$$ represents the loss produced by a mismatch with the governing differential equation $\mathcal{F}$.

Similarly we have the following expression for the boundary loss :

$$\mathcal{L}_{\mathcal{B}}(\theta) = MSE_{\mathcal{B}} = \dfrac{1}{N_B}\sum^{N_B}_{i=1} \left\|\mathcal{B}(\hat{u}_{\theta}(z_i)) - g(z_i)\right\|^2$$

$N_c$ and $N_B$ represent the number of collocation and boundary points, respectively. 

$\mathcal{L}_{data}(\theta)$ is the loss with respect to the data points:

$$\mathcal{L}_{data}(\theta) = MSE_{data} = \dfrac{1}{N_d}\sum^{N_d}_{i=1} \left\|\hat{u}_{\theta}(z_i)) - u^{*}_i\right\|^2$$

Note that the quantity $r_{\mathcal{F}}[\hat{u}_{\theta}](z) = r_{\theta} (z) := \mathcal{F}(\hat{u}_{\theta}(z_i); \gamma) - f(z_i)$ is the differential equation residual. Similarly, the residual NN corresponding to boundary conditions is $r_{\mathcal{B}}[\hat{u}_{\theta}](z) = r_{\theta} (z) := \mathcal{B}(\hat{u}_{\theta}(z)) - g(z)$ [[1](https://doi.org/10.1007/s10915-022-01939-z)].

**Loss function for the Poisson equation**

In our case, we do not consider any data points. Therefore $N_d$ = 0 and $\mathcal{L}_{data}(\theta)=0$. Also we consider for now that $\omega_{\mathcal{F}} = \omega_{\mathcal{B}} = 1$. Finally, we consider in the training a 1-D domain, with $N_c = 1000$ and $N_B = 1$. <font color='red'>Note: Should we consider external boundary condition, and not solely the initial one ?</font>

We note that here, the residual $r_{\theta} (z)$ is as follow :

$$ r_{\theta} (z) = \dfrac{d}{d r}\left(r^2 \dfrac{d \phi}{d r}\right) - \dfrac{2r}{(r+1)^3}$$

Therefore, we want to minimize the following loss function :

$$ \mathcal{L}(\theta) = \dfrac{1}{N_c}\sum^{N_c}_{i=1} \left\|\dfrac{df}{d r} - \dfrac{2r_i}{(r_i+1)^3} \right\|^2 + \dfrac{1}{N_B}\sum^{N_B}_{i=1} \left\|\hat{\Phi}(z_0) - \Phi_0 \right\|^2$$ 

<font color='red'>Unclear to me what the $\mathcal{B}$ operator and the boundary function $g$ are.</font>
where we set $f = r^2 \dfrac{d \phi}{d r}$. 

**Boundary conditions:** We create a domain for which $r \in [a; b]$.
Hernquist shows that by integrating the aforementionned Poisson equation, one gets that:

$$ \Phi(r) = - \dfrac{GM}{r + a}$$

Let us recall that we have set $\frac{GM}{a} = 1$, meaning that $GM=a$. Therefore, in our case we should rewrite the potential such as :

$$ \Phi(r) = - \dfrac{1}{r + a}$$

In [ ]:
import torch
from torch import nn, optim
import numpy as np
from tqdm.notebook import tqdm, trange
from pyDOE import lhs
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')

**Implementation of the Neural Network**

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_hidden: int, dim_hidden: int, act=nn.Tanh()):
        super().__init__()

        self.layer_in = nn.Linear(1, dim_hidden)
        self.layer_out = nn.Linear(dim_hidden, 1)

        num_middle = num_hidden - 1
        self.middle_layers = nn.ModuleList([nn.Linear(dim_hidden, dim_hidden) for _ in range(num_middle)])
        self.act = act

    def forward(self, x):
        out = self.act(self.layer_in(x))
        for layer in self.middle_layers:
            out = self.act(layer(out))
        return self.layer_out(out)

**Implementation of the derivatives for the loss function**

In [ ]:
def pde_residual(net, x):
    phi_pred = net(x)
    phi_x = torch.autograd.grad(phi_pred, x, grad_outputs=torch.ones_like(x), create_graph=True, retain_graph=True,)[0]
    phi_x *= x ** 2
    df = torch.autograd.grad(phi_x, x, grad_outputs=torch.ones_like(x), create_graph=True, retain_graph=True,)[0]
    residual = df - 2 * x / (x + 1) ** 3
    return residual.pow(2).mean()

**Implementation of the loss function**

In [ ]:
def boundary_error(net, x_b, y_true):
    phi_pred = net(x_b)
    error = y_true - phi_pred
    return error.pow(2).mean()

def loss_function(net, x, x_b, y_true):
    return pde_residual(net, x) + boundary_error(net, x_b, y_true)

**Implementation of the training phase**

In [ ]:
class TrainingPhase:
    def __init__(self, neural_net, domain, n_epochs, optimizer, _loss_function, x_train, y_train):
        self.neural_net = neural_net
        self.domain = domain
        self.n_epochs = n_epochs
        self.optimizer = optimizer
        self.loss_function = _loss_function
        self.x_train = x_train
        self.y_true = y_train

    def train_model_general(self, lr, x_val, y_val):
        opt = self.optimizer(self.neural_net.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,
                                                           mode='min',
                                                           factor=0.5,
                                                           patience=1000,
                                                           threshold=0.0001,
                                                           verbose=False)
        loss = self.loss_function

        losses = []
        val_losses = []
        epochs = []
        for epoch in trange(self.n_epochs, desc="Epoch"):

            opt.zero_grad()
            loss_value = loss(self.neural_net, self.domain, self.x_train, self.y_true)
            loss_value.backward()
            opt.step()
            scheduler.step(loss_value)
            
            val_loss = (y_val - self.neural_net(x_val)).pow(2).mean()
            losses.append(loss_value.item())
            val_losses.append(val_loss.item())
        return self.neural_net, np.array(epochs), np.array(losses), np.array(val_losses)
    
    def train_model_LBFGS(self, lr):
    # Remplacez votre optimisateur par L-BFGS
        opt = torch.optim.LBFGS(self.neural_net.parameters(), lr=lr)
        loss = self.loss_function

        losses = []
        val_losses = []
        epochs = []

        # Créer une fonction de fermeture pour calculer la perte.
        def closure():
            opt.zero_grad()
            loss_value = loss(self.neural_net, self.domain, self.x_train, self.y_true)
            loss_value.backward()
            return loss_value

        for epoch in trange(self.n_epochs, desc="Epoch"):
            # L'optimisation L-BFGS requiert l'appel à la méthode step avec la fermeture de la fonction de perte
            opt.step(closure)
            current_loss = closure()
            val_loss = (y_val - self.neural_net(x_val)).pow(2).mean()
            losses.append(current_loss.item())
            val_losses.append(val_loss.item())

        return self.neural_net, np.array(epochs), np.array(losses), np.array(val_losses)
    
    def train_model(self, lr, x_val, y_val):
        if self.optimizer == 'LBFGS':
            print("Training with LBFGS optimizer")
            net, epochs, losses, val_losses = self.train_model_LBFGS(lr, x_val, y_val)
        else:
            net, epochs, losses, val_losses = self.train_model_general(lr, x_val, y_val)
        return net, epochs, losses, val_losses
    
    def save_model(self, filename):
        torch.save(self.neural_net, filename)

**Training the Model**

In [ ]:
def analytic_prediction(r):
    """ Value of the gravitational potential 
        in the case of an Hernquist profile 
    """
    return -1/(r + 1)

**Generating Training Data**

In [ ]:
# Initial conditions
Npoints = 10_000
Nf = 1024  # collocation points
Nvalidation = Nf // 3
x0 = 0.0
f0 = analytic_prediction(x0)
xf = 1000
ff = analytic_prediction(xf)
# Domain
torch.manual_seed(0)
s = torch.linspace(x0, xf, Npoints)
s_val = (x0 + (xf - x0) * torch.rand(Nvalidation)).reshape(-1, 1)
phi_val = analytic_prediction(s_val)

S_train_Nf = x0 + (xf - x0) * lhs(1, Nf)

x_train = torch.Tensor([x0, xf]).reshape(-1, 1)
y_train = torch.Tensor([f0, ff]).reshape(-1, 1)

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("Configuration of training points")
plt.plot(S_train_Nf, np.zeros_like(S_train_Nf), 'b.', label='Collocation Points')
plt.plot(x_train, [0, 0], 'rx', label='Boundary Points')
# Ajouter du texte avec les coordonnées (x_train, y_train)
for i in range(len(x_train)):
    plt.text(x_train[i]-100, 0.05, f'({float(x_train[i]): .3f}, {float(y_train[i]):.3f})')

plt.xlabel('$s$')
plt.xlim((-100, 1150))
plt.ylim((-0.5, 0.5))
plt.legend()
plt.show()

In [ ]:
s_val.shape

In [ ]:
S_train_Nf = torch.Tensor(S_train_Nf)
S_train_Nf.requires_grad = True

**Training**

In [ ]:
# PARAMETERS
n_epochs = 100_000
a = 1.0
NUM_HIDDEN_LAYERS = 2
DIM_HIDDEN_LAYERS = 32
 
pinn = NeuralNet(num_hidden=NUM_HIDDEN_LAYERS, dim_hidden=DIM_HIDDEN_LAYERS, act=nn.Tanh())
print("Training Neural Network :", pinn)



training = TrainingPhase(neural_net=pinn, domain=S_train_Nf, n_epochs=n_epochs, 
                         optimizer=torch.optim.Adam,_loss_function=loss_function, x_train=x_train, y_train=y_train)

net, epochs, losses, val_losses = training.train_model(lr=1e-5, x_val=s_val, y_val=phi_val)
# training.save_model(f"resultats/Hernquist_{n_epochs}.pt")

In [ ]:
testing_domain = torch.linspace(0, 100, 15000, requires_grad=False) * a
y_true = analytic_prediction(testing_domain.numpy())
y_pred_plot = net(testing_domain.reshape(-1, 1)).ravel()

**Plotting the results**  
This part can be run independently from the rest of the notebook, if the model has already been trained once. Indeed the results are saved so the model does not have to be trained each time. To do so, go below to the "[Plotting using pre-trained model" section](#Plotting-using-pre-trained-model).

In [ ]:
plt.figure()
plt.title(f"Hernquist model")
plt.plot(testing_domain.numpy(), y_true, 'b', label="Analytical Value")
plt.plot(testing_domain.numpy(), y_pred_plot.detach().numpy(), '--r', label="Predicted Value")
plt.xlabel('$s$')
plt.ylabel('$\Phi(s)$')
# plt.plot(s.detach().numpy(), y_pred_train.detach().numpy(),'.k', label="Predicted 2")
# plt.savefig(f"resultats/Predicttion_{NUM_HIDDEN_LAYERS}x{DIM_HIDDEN_LAYERS}_linear.png")
plt.legend();

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(net)

In [ ]:
(32 + 32) + (32 * 32 + 32) + (32 + 1)

In [ ]:
def relative_error(domain, y_pred, y_true, plot=False, save=False):
    result = (y_pred - y_true) / y_true
    print(np.mean(result))
    if not plot:
        return result

    plt.figure()
    plt.title(r"Relative error ")
    plt.plot(domain, result)
    plt.xlabel('$s$')
    plt.ylabel('$\dfrac{\Delta \phi}{\phi}$')
    if save:
            plt.savefig(f"Relative_error_hernquist_{a=}.png")
    plt.show()
    return result

In [ ]:
relat_err = relative_error(domain=testing_domain, y_pred=y_pred_plot.detach().numpy(), y_true=y_true, plot=True, save=True)

In [ ]:
plt.figure()
plt.title("Loss Function")
plt.plot(np.arange(n_epochs), losses, 'k', label="Training Loss")
plt.plot(np.arange(n_epochs), val_losses, 'r', label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss value")
plt.yscale("log")
plt.legend();
#plt.savefig(f"resultats/Loss_{NUM_HIDDEN_LAYERS}x{DIM_HIDDEN_LAYERS}_log.png")

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

# Définition de la fonction qui représente le système d'équations différentielles
def system(s, y):
    return [y[1], (2*s/(s+1)**3) / s**2]

# Conditions initiales
s0 = 0.01 # Evite division par 0, à ajuster selon votre problème
Phi_prime0 = 0 # Phi'(0)
Phi_double_prime0 = -1 # Phi''(0)

y0 = [Phi_prime0, Phi_double_prime0]

# Grille pour la solution
s = np.linspace(s0, 10, 1000)  # Ajustez selon vos besoins

In [ ]:
%%time
# Résolution du système d'équations différentielles
sol = solve_ivp(system, [s0, 10], y0, t_eval=s, method='RK45')

In [ ]:
plt.plot(s, sol.y[1], label="Computed value")
plt.plot(s, analytic_prediction(s), '--r', label="Analytical Value")
plt.legend();

In [ ]:
array = testing_domain.reshape(-1, 1)

In [ ]:
%%timeit

net(array)